# <center>Recupération des données </center>
# <center>et mise en base de données </center>

In [1]:
# Import des librairies pour le Scrapping
import requests
from bs4 import BeautifulSoup
import lxml.html

___

In [2]:
# Import pour la gestion de la BDD
import sqlite3   # BDD pour le MOC

___

In [3]:
import pandas as pd
import string
import re
import sys

NB_Pages=2#2032
liensBD=[]

___

In [4]:
from datetime import datetime # Module pour la gestion temporelle
import time

___

In [5]:
def heure():
    x = datetime.now()
    return x.strftime("%m"),x.strftime("%d"),x.strftime("%H"),x.strftime("%M")

In [6]:
# Critiques de BD:
for idx in range (1, NB_Pages):
    if (idx%800!=0 or idx==1341):
        dom = lxml.html.fromstring(requests.get('https://www.babelio.com/livres-/bande-dessinee/18?page='+str(idx)).content)
        liensBD+=[x for x in dom.xpath('//a/@href') if '/livres/' in x and ('#c') not in x]
        Heure=heure()
        print(Heure[2],":",Heure[3],"Pages ",str(idx)," sur ",str(NB_Pages))
        time.sleep(9)
    else :
        time.sleep(600)

13 : 47 Pages  1  sur  2


In [7]:
dfl = pd.DataFrame(liensBD)
dfl=dfl.rename(columns = {0:'liens'})
# On garde une copie des liens récupérer sur le site babelio.com
# On ne sauvegrde que la partie
dfl.to_csv('CSV_tmp/list.csv',sep="\t")

In [8]:
dfl=pd.read_csv('CSV_tmp/list.csv',sep="\t")
dfl

,Unnamed: 0,liens
0,0,/livres/Herge-Les-Aventures-de-Tintin-tome-03-...
1,1,/livres/Herge-Les-Aventures-de-Tintin-tome-11-...
2,2,/livres/Herge-Les-Aventures-de-Tintin-tome-17-...
3,3,/livres/Herge-Les-aventures-de-Tintin-Tome-6--...
4,4,/livres/Herge-Les-aventures-de-Tintin-tome-16-...
...,...,...
95,95,/livres/Marjane-Persepolis/3934
96,96,/livres/Fabcaro-Zai-zai-zai-zai/719553
97,97,/livres/Morris-Lucky-Luke-tome-8--Chasseur-de-...
98,98,/livres/Mermoux-Linvitation/892315


In [9]:
dfl=dfl.drop('Unnamed: 0',axis=1) # On retire la colonne inutile

In [10]:
dfl

,liens
0,/livres/Herge-Les-Aventures-de-Tintin-tome-03-...
1,/livres/Herge-Les-Aventures-de-Tintin-tome-11-...
2,/livres/Herge-Les-Aventures-de-Tintin-tome-17-...
3,/livres/Herge-Les-aventures-de-Tintin-Tome-6--...
4,/livres/Herge-Les-aventures-de-Tintin-tome-16-...
...,...
95,/livres/Marjane-Persepolis/3934
96,/livres/Fabcaro-Zai-zai-zai-zai/719553
97,/livres/Morris-Lucky-Luke-tome-8--Chasseur-de-...
98,/livres/Mermoux-Linvitation/892315


In [11]:
liensBD=dfl['liens']

In [12]:
# Connection à la BDD
connecteur=sqlite3.connect('LesBD.sq3')
# Création d'un curseur
curseur=connecteur.cursor()
# Création des tables
curseur.execute("""CREATE TABLE BD(
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    Titre text NOT NULL,
                    EAN int,
                    Resume text,
                    Auteur text,
                    Image text,
                    Dessinateur null)
                    """)
# Sauvegarde des modifixations
connecteur.commit()
# Fermeture de la connection
connecteur.close()

In [13]:
def Changer_type(l, dtype=str):
    return list(map(dtype, l))


def remove_tags(text) :
    TAG_RE=re.sub('\n |\t|\n', '', str(text))
    return TAG_RE

In [14]:
# POUR LE POC ON SE CONCENTRE SUR L'AUTEUR PRINCIPAL
# CETTE PARTIE DE CODE ALIMENTE SEULEUMENT LA PARTIE LIVRE ET SA DESCRIPTION
idx=0
connecteur=sqlite3.connect('LesBD.sq3')
curseur=connecteur.cursor()
print("Début du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])
for url_BD in liensBD:
    
    try:
            idx+=1
            LinksList=[]
            Heure=heure()
            #print("Le ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3],"|| album numéro ",str(idx) , ": ",str(url_BD))
            response_BD=requests.post("https://www.babelio.com/"+url_BD)
            soup = BeautifulSoup(response_BD.text)
            EAN=soup.find('div',class_="livre_refs grey_light").text
            EAN=EAN[9:23]
            #print(EAN)
            Title=soup.find("h1", itemprop="name").text
            Title=remove_tags(Title)
            #print(Title)
            Image=soup.find("div",class_="col col-4")
            img_src = Image.find('img').attrs['src']
            Auteur1=soup.find("span", itemprop="name").text
            #print(Auteur1)
            Dessinateur=""
            Description=[des for des in (soup.find("div", itemprop="description",class_="livre_resume")) if des!= '[]']
            Description=remove_tags(Description)
            #print(Description)  
            Rating=soup.find("span", itemprop="ratingValue").text               
            #print(Rating)
            time.sleep(8)
            print("Sauvegarde en BDD de ",url_BD)
            curseur.execute("INSERT INTO BD (Titre,EAN,Resume,Auteur,Image) VALUES (?, ?, ?, ?,?)",(Title, EAN, Description,Auteur1,img_src))
            connecteur.commit()
    except Exception as e:
            print("Erreur...",e,"\n")
            print("Sur le livre ",url_BD)
    
connecteur.close()
print("Fin du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])

Début du scrapping:  14   09  à  13 : 47
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-03--Tintin-en-Ameriq/401917
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-11--Le-Secret-de-La-/403087
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-17--On-a-marche-sur-/403091
Sauvegarde en BDD de  /livres/Herge-Les-aventures-de-Tintin-Tome-6--LOreille-cassee/402192
Sauvegarde en BDD de  /livres/Herge-Les-aventures-de-Tintin-tome-16--Objectif-lune/3945
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-04--Les-Cigares-du-p/401918
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-12--Le-Tresor-de-Rac/403088
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-05--Le-Lotus-bleu/401921
Sauvegarde en BDD de  /livres/Herge-Les-Aventures-de-Tintin-tome-07--Lle-Noire/402195
Sauvegarde en BDD de  /livres/Herge-Les-aventures-de-Tintin-tome-22--Vol-714-pour-Syd/3951
Sauvegarde en BDD de  /livres/Goscinny-Asterix-t

In [15]:
# Connection à la BDD
connecteur=sqlite3.connect('LesBD.sq3')
# Création d'un curseur
curseur=connecteur.cursor()
# Création des tables
curseur.execute("""CREATE TABLE CRITIQUES(
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    Titre text NOT NULL,
                    Note int,
                    Critique text,
                    Auteur text)
                    """)
# Sauvegarde des modifixations
connecteur.commit()
# Fermeture de la connection
connecteur.close()

In [19]:
# CETTE PARTIE DU SCRIPT SAUVE EN BDD LES CRITIQUES
connecteur=sqlite3.connect('LesBD.sq3')
curseur=connecteur.cursor()
print("Début du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])
for url_BD in liensBD:
        idx=1
        nbpacri=15 # Nb de pages ce critiques entré en dur pour l'instant
        while idx<nbpacri:
            tmp="https://www.babelio.com"+str(url_BD)+"/critiques?a=a&pageN="+str(idx)
            #print(tmp)
            response_BD=requests.post(tmp)
            try:
                soup= BeautifulSoup(response_BD.text)
                
                Critiques=[critique.text for critique in  soup.find_all("div", id=re.compile('^cri_'))]
                #print(Critiques)
                Title=soup.find("h1", itemprop="name").text
                Title=remove_tags(Title)
                #print(Title)
                Lst_Lecteurs=[lecteur.text for lecteur in soup.find_all("span", itemprop="name")]
                #print(Lst_Lecteurs)
                Lst_Notes=soup.find_all(itemprop={"ratingValue","content"})
                Lst_Notes=([float(s) for s in re.findall(r'-?\d+\.?\d*', str(Lst_Notes))])
                del Lst_Notes[0]  # On supprime le 1er élément qui correspond à la note moyenne de la BD
                #print(Lst-Notes)
                for num,lecteur in enumerate(Lst_Lecteurs): # On a une critique par lecteur
                    curseur.execute("INSERT INTO CRITIQUES (Titre,Note,Critique,Auteur) VALUES (?,?,?,?)",(Title, Lst_Notes[num], Critiques[num],lecteur))
                    connecteur.commit()
            except Exception as e:
                #print(e)
                pass
            finally:
                idx+=1
print("Fin du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])            
        

Début du scrapping:  14   09  à  14 : 01
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range